In [101]:
import time
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import pandas as pd
from pandas import DataFrame
from geopy.geocoders import Photon
import json
from flask import Flask, jsonify, request

In [22]:
# function that scrapes shelter data and stores in a dictionary
def get_shelters(zip_code, my_dict):
    url = 'https://www.hud.gov/findshelter'
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(1)
    search = driver.find_element(By.XPATH, '//*[@id="place"]')
    search.click()
    time.sleep(.5)
    search.send_keys(zip_code)
    time.sleep(.5)
    send = driver.find_element(By.XPATH, '//*[@id="hero"]/div[2]/div/form/div/div[2]/input')
    send.click()
    time.sleep(.5)
    results = driver.find_elements(By.XPATH, '//*[@id="results"]/li')
    for result in results:
        name = result.find_element(By.TAG_NAME, 'h6').text
        addr = result.find_element(By.CLASS_NAME, 'address').text
        try:
            web = result.find_element(By.CLASS_NAME, 'web').text
        except:
            web = 'N/A'

        try:
            phone = result.find_element(By.CLASS_NAME, 'phone').text
        except:
            phone = 'N/A'
        my_dict[name] = [addr, web, phone]
    driver.close()

In [28]:
## build database containing all Portland shelters
# get all zipcodes *blocked by web
pdx_zip = []
url = 'https://www.unitedstateszipcodes.org/or/#zips-list'
time.sleep(1)
driver = webdriver.Chrome()
driver.get(url)
time.sleep(1)
cities = driver.find_elements(By.CLASS_NAME, 'row')
time.sleep(1)
for city in cities:
    if city.find_element(By.CLASS_NAME, 'col-xs-12 prefix-col3').text == 'Portland':
        pdx_zip.append(city.find_element(By.CLASS_NAME, 'col-xs-12 prefix-col1').text.int())
        time.sleep(.5)
pdx_zip

[]

In [34]:
# get all zipcodes
url = 'https://zipcode.org/city/OR/PORTLAND'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

data = soup.find_all(class_='List_Link_Text')
for link in data:
    text = link.get_text()

    if 'Zip' in text:
        zip_code = text.split()[0]
        pdx_zip.append(zip_code)

['97201',
 '97202',
 '97203',
 '97204',
 '97205',
 '97206',
 '97207',
 '97208',
 '97209',
 '97210',
 '97211',
 '97212',
 '97213',
 '97214',
 '97220',
 '97221',
 '97215',
 '97216',
 '97217',
 '97218',
 '97219',
 '97222',
 '97223',
 '97227',
 '97228',
 '97229',
 '97231',
 '97232',
 '97233',
 '97236',
 '97256',
 '97258',
 '97266',
 '97268',
 '97269',
 '97280',
 '97290',
 '97291',
 '97292',
 '97293',
 '97224',
 '97225',
 '97230',
 '97238',
 '97239',
 '97240',
 '97242',
 '97267',
 '97281',
 '97282',
 '97283',
 '97286',
 '97294',
 '97296',
 '97298']

In [36]:
# get all shelters in pdx (34)
for zip_ in pdx_zip:
    get_shelters(zip_, pdx)

In [48]:
df = pandas.DataFrame(pdx)
df

,Family Promise of Greater Washington County,Family Promise of Beaverton,Path Home,Family Promise of Metro East,Good Neighbor Center,Transition Projects Laurelwood Center Shelter,Kenton Women's Village (Argyle Village),Right 2 Dream Too,Greater New Hope Family Services,Dignity Village,...,Share Homestead,Menlo Park Safe Rest Village,Transition Projects Doreen's Place Shelter,Our Just Future (formerly Human Solutions),Blanchet House,Emergency Shelter Clearing House,Share Aspire Program,Transition Projects Resource Center,Reedway Safe Rest Village,Our Just Future
0,"5625 Southwest Erickson Avenue, Beaverton","14986 Northwest Cornell Road, Portland","6220 Southeast 92nd Avenue, Portland","4837 Northeast Couch Street, Portland","11130 Southwest Greenburg Road, Tigard","6130 Southeast Foster Road, Portland","2250-2344 North Columbia Boulevard, Portland","999 North Thunderbird Way, Portland","11936 Northeast Sandy Boulevard, Portland","9401 Northeast Sunderland Avenue, Portland",...,"4921 Northeast Hazel Dell Avenue, Vancouver","12202 East Burnside Street, Portland","610 Northwest Broadway, Portland","124 Northeast 181st Avenue Suite 109, Portland","310 Northwest Glisan Street, Portland",Vancouver,"2306 Northeast Andresen Road, Vancouver","650 Northwest Irving Street, Portland","10550 Southeast Reedway Street, Portland","10550 Northeast Halsey Street, Portland"
1,https://www.familypromisegwc.org/?utm_source=g...,https://www.familypromisegwc.org/,http://www.path-home.org/,https://www.familypromisemetroeast.org/about-u...,http://gncnw.org/,https://www.tprojects.org/contact-us,https://www.catholiccharitiesoregon.org/servic...,http://right2dreamtoo.blogspot.com/,http://www.greaternewhopefamilyservices.com/co...,https://dignityvillage.org/,...,https://sharevancouver.org/programs/share-shel...,https://www.portland.gov/ryan/menlo-park-srv-1...,https://www.tprojects.org/contact-us,http://www.humansolutions.org/,http://blanchethouse.org/,N/A,http://www.sharevancouver.org/,https://www.tprojects.org/resource-center/,https://www.portland.gov/safe-rest-villages/lo...,http://www.humansolutions.org/
2,(971) 217-8949,(971) 217-8949,(503) 915-8306,(503) 753-3960,(503) 443-6084,(503) 280-4776,(503) 231-4866,(503) 382-8838,(813) 417-4013,(503) 281-1604,...,(360) 693-8923,(503) 865-6407,(503) 280-4664,(503) 405-7875,(503) 241-4340,(360) 695-9677,(360) 448-2121,(503) 280-4700,N/A,(503) 548-0200


In [51]:
data = []
for shelter in pdx:
    info = {}
    info['name'] = shelter
    info['addr'] = pdx[shelter][0]
    info['web'] = pdx[shelter][1]
    info['phone'] = pdx[shelter][2]
    data.append(info)
data

[{'name': 'Family Promise of Greater Washington County',
  'addr': '5625 Southwest Erickson Avenue, Beaverton',
  'web': 'https://www.familypromisegwc.org/?utm_source=google&utm_medium=wix_google_business_profile&utm_campaign=17066741240387882471',
  'phone': '(971) 217-8949'},
 {'name': 'Family Promise of Beaverton',
  'addr': '14986 Northwest Cornell Road, Portland',
  'web': 'https://www.familypromisegwc.org/',
  'phone': '(971) 217-8949'},
 {'name': 'Path Home',
  'addr': '6220 Southeast 92nd Avenue, Portland',
  'web': 'http://www.path-home.org/',
  'phone': '(503) 915-8306'},
 {'name': 'Family Promise of Metro East',
  'addr': '4837 Northeast Couch Street, Portland',
  'web': 'https://www.familypromisemetroeast.org/about-us.html',
  'phone': '(503) 753-3960'},
 {'name': 'Good Neighbor Center',
  'addr': '11130 Southwest Greenburg Road, Tigard',
  'web': 'http://gncnw.org/',
  'phone': '(503) 443-6084'},
 {'name': 'Transition Projects Laurelwood Center Shelter',
  'addr': '6130 So

In [52]:
df = pandas.DataFrame(data)

In [93]:
geolocator = Photon(user_agent="geoapi")
for i in range(34):
    df.loc[i, "latitude"], df.loc[i, "longitude"] = geolocator.geocode(df.loc[i, "addr"])[-1][0:2]

In [100]:
df = df.drop('latitude', axis = 0)
df = df.drop('longitude', axis = 0)
df

,name,addr,web,phone,latitude,longitude
0,Family Promise of Greater Washington County,"5625 Southwest Erickson Avenue, Beaverton",https://www.familypromisegwc.org/?utm_source=g...,(971) 217-8949,45.478980,-122.812407
1,Family Promise of Beaverton,"14986 Northwest Cornell Road, Portland",https://www.familypromisegwc.org/,(971) 217-8949,45.571229,-122.831362
2,Path Home,"6220 Southeast 92nd Avenue, Portland",http://www.path-home.org/,(503) 915-8306,45.477379,-122.619253
3,Family Promise of Metro East,"4837 Northeast Couch Street, Portland",https://www.familypromisemetroeast.org/about-u...,(503) 753-3960,45.524134,-122.613245
4,Good Neighbor Center,"11130 Southwest Greenburg Road, Tigard",http://gncnw.org/,(503) 443-6084,45.439764,-122.772712
5,Transition Projects Laurelwood Center Shelter,"6130 Southeast Foster Road, Portland",https://www.tprojects.org/contact-us,(503) 280-4776,45.493899,-122.605208
6,Kenton Women's Village (Argyle Village),"2250-2344 North Columbia Boulevard, Portland",https://www.catholiccharitiesoregon.org/servic...,(503) 231-4866,45.575717,-122.642670
7,Right 2 Dream Too,"999 North Thunderbird Way, Portland",http://right2dreamtoo.blogspot.com/,(503) 382-8838,45.530076,-122.668893
8,Greater New Hope Family Services,"11936 Northeast Sandy Boulevard, Portland",http://www.greaternewhopefamilyservices.com/co...,(813) 417-4013,45.525056,-122.648124
9,Dignity Village,"9401 Northeast Sunderland Avenue, Portland",https://dignityvillage.org/,(503) 281-1604,45.590597,-122.635763


In [127]:
df.T.to_json()

'{"0":{"name":"Family Promise of Greater Washington County","addr":"5625 Southwest Erickson Avenue, Beaverton","web":"https:\\/\\/www.familypromisegwc.org\\/?utm_source=google&utm_medium=wix_google_business_profile&utm_campaign=17066741240387882471","phone":"(971) 217-8949","latitude":45.4789802,"longitude":-122.8124068688},"1":{"name":"Family Promise of Beaverton","addr":"14986 Northwest Cornell Road, Portland","web":"https:\\/\\/www.familypromisegwc.org\\/","phone":"(971) 217-8949","latitude":45.5712294,"longitude":-122.8313623086},"2":{"name":"Path Home","addr":"6220 Southeast 92nd Avenue, Portland","web":"http:\\/\\/www.path-home.org\\/","phone":"(503) 915-8306","latitude":45.47737905,"longitude":-122.6192527253},"3":{"name":"Family Promise of Metro East","addr":"4837 Northeast Couch Street, Portland","web":"https:\\/\\/www.familypromisemetroeast.org\\/about-us.html","phone":"(503) 753-3960","latitude":45.5241343,"longitude":-122.6132448206},"4":{"name":"Good Neighbor Center","addr

In [110]:
geolocator.geocode("97223 OR")

Location(Southwest Orchid Street, 97223, Oregon, United States, (45.4581464, -122.7440512, 0.0))

In [125]:
# dictionary with all PDX zip codes and corresponding lat and long
pdx_coord = {}
driver = webdriver.Chrome()
url = 'https://www.freemaptools.com/convert-us-zip-code-to-lat-lng.htm'
driver.get(url)
time.sleep(1)
box = driver.find_element(By.XPATH, '//*[@id="postcode"]')
search = driver.find_element(By.XPATH, '//*[@id="content"]/center/p')
for i in pdx_zip:
    box.click()
    box.clear()
    time.sleep(.5)
    box.send_keys(i)
    time.sleep(.5)
    search.click()
    time.sleep(.5)
    try:
        lat, long = driver.find_element(By.XPATH, '//*[@id="divFeedback"]/strong').text.split(',')
        pdx_coord[i] = (lat,long)
    except:
        pdx_coord[i] = "N/A"
driver.close()
pdx_coord

{'97201': ('45.50742', '-122.68984'),
 '97202': ('45.48031', '-122.64512'),
 '97203': ('45.61112', '-122.74134'),
 '97204': ('45.51815', '-122.67415'),
 '97205': ('45.51548', '-122.69877'),
 '97206': ('45.47993', '-122.60057'),
 '97207': ('45.52390', '-122.67510'),
 '97208': ('45.52390', '-122.67510'),
 '97209': ('45.53346', '-122.68150'),
 '97210': ('45.55154', '-122.73514'),
 '97211': ('45.57732', '-122.64162'),
 '97212': ('45.54417', '-122.64307'),
 '97213': ('45.53779', '-122.60081'),
 '97214': ('45.51394', '-122.64412'),
 '97220': ('45.55191', '-122.55533'),
 '97221': ('45.49829', '-122.72776'),
 '97215': ('45.51396', '-122.59940'),
 '97216': ('45.51342', '-122.55834'),
 '97217': ('45.58959', '-122.69299'),
 '97218': ('45.58088', '-122.60030'),
 '97219': ('45.45102', '-122.69501'),
 '97222': ('45.44218', '-122.61861'),
 '97223': ('45.44686', '-122.79557'),
 '97227': ('45.54028', '-122.67696'),
 '97228': ('45.52390', '-122.67510'),
 '97229': ('45.55438', '-122.81131'),
 '97231': ('

In [121]:
pdx_coord

{'97201': ('45.50742', '-122.68984'),
 '97202': ('45.48031', '-122.64512'),
 '97203': ('45.61112', '-122.74134'),
 '97204': ('45.51815', '-122.67415'),
 '97205': ('45.51548', '-122.69877'),
 '97206': ('45.47993', '-122.60057'),
 '97207': ('45.52390', '-122.67510'),
 '97208': ('45.52390', '-122.67510'),
 '97209': ('45.53346', '-122.68150'),
 '97210': ('45.55154', '-122.73514'),
 '97211': ('45.57732', '-122.64162'),
 '97212': ('45.54417', '-122.64307'),
 '97213': ('45.53779', '-122.60081'),
 '97214': ('45.51394', '-122.64412'),
 '97220': ('45.55191', '-122.55533'),
 '97221': ('45.49829', '-122.72776'),
 '97215': ('45.51396', '-122.59940'),
 '97216': ('45.51342', '-122.55834'),
 '97217': ('45.58959', '-122.69299'),
 '97218': ('45.58088', '-122.60030'),
 '97219': ('45.45102', '-122.69501'),
 '97222': ('45.44218', '-122.61861'),
 '97223': ('45.44686', '-122.79557'),
 '97227': ('45.54028', '-122.67696'),
 '97228': ('45.52390', '-122.67510'),
 '97229': ('45.55438', '-122.81131'),
 '97231': ('

In [122]:
len(pdx_zip)

55